In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import xgboost as xgb
import joblib

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
data_path = '/content/gdrive/My Drive/ENERGY_CLIMATE_HACK/raw_data/'
df = pd.read_csv(f'{data_path}clean_data.csv')

In [ ]:
## Change to relative terms
abs_vars = ["permanent_resident_population", "swiss_citizen", "foreign_citizen", "age_0to14", "age_15to19", "age_20to39", "age_40to59", "age_60to64", "age_65tomore", "men", "women"]
rel_vars = ["permanent_resident_population_%", "swiss_citizen_%", "foreign_citizen_%", "age_0to14_%", "age_15to19_%", "age_20to39_%", "age_40to59_%", "age_60to64_%", "age_65tomore_%", "men_%"]
drop_vars = ["permanent_resident_population", "swiss_citizen", "foreign_citizen", "age_0to14", "age_15to19", "age_20to39", "age_40to59", "age_60to64", "age_65tomore", "men", "women", 
             "Unnamed: 0", "region_id", "kanton_number",	"kanton_short", "district_number", "temp_stdev", "pv_out_std", "municipality_type", "region_name"]


In [ ]:
def var_clean(absolute_vars, relative_vars, drop_vars, df):
  """
  Inputs: the variables in absolute terms, the var names you want in relative terms
  Output: dataframe with relative vars and dropped unncessary vars 
  """
  for a, r in zip(absolute_vars, relative_vars):
    df[r] = df[a]/df["population"]

  for i in drop_vars:
    df = df.drop([i], axis = 1)
  return df
df = var_clean(abs_vars, rel_vars, drop_vars, df) 
df.head(3)

,percentage_population_in_detached_houses,population,population_per_km2,percentage_detached_houses,percentage_flats_3to4_rooms,district_name,kanton_name,taxable_income_in_mio_CHF,taxable_income_per_taxable_person_in_chf,share_electric_cars,share_pv,share_renewable_heating,Heating data coverage,mean_electricity_tarif_2021_rp_per_kwh,% Ja,% participation,vote_accepted,vote_bins,temp_mean,pv_out_mea,NUMPOINTS,permanent_resident_population_%,swiss_citizen_%,foreign_citizen_%,age_0to14_%,age_15to19_%,age_20to39_%,age_40to59_%,age_60to64_%,age_65tomore_%,men_%
0,49.555336,1981,250.442478,69.565217,41.900648,Affoltern,Zürich,94.4072,105013.57060,0.02731,0.07149,0.6129,0.9928,15.486667,55.41,59.46,1,marginally for,9.992784,3.248851,15,1.000000,0.872287,0.127713,0.155982,0.062090,0.197375,0.357900,0.065119,0.161535,0.498233
1,42.616626,5721,758.753316,72.784314,52.324195,Affoltern,Zürich,158.0976,75105.74822,0.01728,0.04280,0.4325,0.7072,15.486667,55.41,59.46,1,marginally for,9.992784,3.248851,15,0.875546,0.703723,0.171823,0.154344,0.049467,0.202237,0.272680,0.053662,0.143157,0.437161
2,36.837533,2293,177.889837,58.333333,45.512821,Dielsdorf,Zürich,74.0686,73920.75848,0.01727,0.02390,0.4191,0.9885,15.486667,55.41,59.46,1,marginally for,10.088192,3.238797,17,0.961622,0.830789,0.130833,0.150458,0.056694,0.225469,0.321413,0.061491,0.146097,0.478849


In [ ]:
def one_hot(df):
  """
  One hot encoding, with drop_first option (interpret in terms of marginally against)
  args: 
  df = orginal df
  """

  one_hot = pd.get_dummies(df['vote_bins'], drop_first= True)
  # Drop column vote_bins as it is now encoded
  df = df.drop('vote_bins',axis = 1)
  # Join the encoded df
  df = df.join(one_hot)
  return df
df = one_hot(df)



In [ ]:
## Econometrics section:
import sklearn
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
for i in df.columns:
  print(i)

percentage_population_in_detached_houses
population
population_per_km2
percentage_detached_houses
percentage_flats_3to4_rooms
district_name
kanton_name
taxable_income_in_mio_CHF
taxable_income_per_taxable_person_in_chf
share_electric_cars
share_pv
share_renewable_heating
Heating data coverage
mean_electricity_tarif_2021_rp_per_kwh
% Ja
% participation
vote_accepted
temp_mean
pv_out_mea
NUMPOINTS
permanent_resident_population_%
swiss_citizen_%
foreign_citizen_%
age_0to14_%
age_15to19_%
age_20to39_%
age_40to59_%
age_60to64_%
age_65tomore_%
men_%
marginally for
strongly against
strongly for


#### Basic OLS Regression 

In [ ]:
target_vars = ["share_electric_cars", "share_pv", "share_renewable_heating"]
exog_vars_pv = ["percentage_population_in_detached_houses", "population_per_km2", "percentage_flats_3to4_rooms", "taxable_income_per_taxable_person_in_chf",
"mean_electricity_tarif_2021_rp_per_kwh", "% participation", "temp_mean", "pv_out_mea", "permanent_resident_population_%", "swiss_citizen_%", "age_0to14_%", "age_15to19_%", "age_20to39_%",
"age_40to59_%", "age_60to64_%", "age_65tomore_%", "men_%", "marginally for", "strongly against", "strongly for"]
exog_vars_heat = ["percentage_population_in_detached_houses", "population_per_km2", "percentage_flats_3to4_rooms", "taxable_income_per_taxable_person_in_chf", "Heating data coverage",
"mean_electricity_tarif_2021_rp_per_kwh", "% participation", "temp_mean", "pv_out_mea", "permanent_resident_population_%", "swiss_citizen_%", "age_0to14_%", "age_15to19_%", "age_20to39_%",
"age_40to59_%", "age_60to64_%", "age_65tomore_%", "men_%", "marginally for", "strongly against", "strongly for"]
exog_vars_ev = ["percentage_population_in_detached_houses", "population_per_km2", "percentage_flats_3to4_rooms", "taxable_income_per_taxable_person_in_chf", "mean_electricity_tarif_2021_rp_per_kwh", "% participation", "temp_mean", "permanent_resident_population_%", "swiss_citizen_%", "age_0to14_%", "age_15to19_%", "age_20to39_%",
"age_40to59_%", "age_60to64_%", "age_65tomore_%", "men_%", "marginally for", "strongly against", "strongly for", "NUMPOINTS",]

##### Electric Car Share

In [ ]:
endo_var = "share_electric_cars"
exog_vars = ["percentage_population_in_detached_houses", "population_per_km2", "percentage_flats_3to4_rooms", "taxable_income_per_taxable_person_in_chf", "mean_electricity_tarif_2021_rp_per_kwh", "% participation", "temp_mean", "permanent_resident_population_%", "swiss_citizen_%", "age_0to14_%", "age_15to19_%", "age_20to39_%",
"age_40to59_%", "age_60to64_%", "age_65tomore_%", "men_%", "marginally for", "strongly against", "strongly for", "NUMPOINTS",]
def ols_reg(X,y): 
 
    X = sm.add_constant(X) 

    model = sm.OLS(y, X).fit() 
    predictions = model.predict(X)

    # Print out the statistics
    return print(model.summary())
ols_reg(X = df[exog_vars], y = df[endo_var])

                             OLS Regression Results                            
Dep. Variable:     share_electric_cars   R-squared:                       0.205
Model:                             OLS   Adj. R-squared:                  0.197
Method:                  Least Squares   F-statistic:                     27.14
Date:                 Fri, 03 Sep 2021   Prob (F-statistic):           9.51e-86
Time:                         09:05:27   Log-Likelihood:                 7482.2
No. Observations:                 2021   AIC:                        -1.492e+04
Df Residuals:                     2001   BIC:                        -1.481e+04
Df Model:                           19                                         
Covariance Type:             nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------

Analysis EVs - 
significant variables at 5% level of significance:
1. detatched houses
2. pop density 
3. 3 to floor rooms
4. income
5. electricity tariff
6. 15-19
7. 20-39
8. 40-59
9. 60-64
10. voting: marginally for increases EV share 

not significant:
1. % participation
2. temperature
3. permanent resident population 
4. swiss_citizen
5. age_0to14%_%
6. 65 and above 
7. men


In [ ]:
#### log evs  
endo_var = "share_electric_cars"
exog_vars = ["percentage_population_in_detached_houses", "population_per_km2", "percentage_flats_3to4_rooms", "taxable_income_per_taxable_person_in_chf", "mean_electricity_tarif_2021_rp_per_kwh", "% participation", "temp_mean", "permanent_resident_population_%", "swiss_citizen_%", "age_0to14_%", "age_15to19_%", "age_20to39_%",
"age_40to59_%", "age_60to64_%", "age_65tomore_%", "men_%", "marginally for", "strongly against", "strongly for", "NUMPOINTS",]
ols_reg(X = df[exog_vars], y = np.log(df[endo_var]))

                             OLS Regression Results                            
Dep. Variable:     share_electric_cars   R-squared:                       0.220
Model:                             OLS   Adj. R-squared:                  0.213
Method:                  Least Squares   F-statistic:                     29.76
Date:                 Fri, 03 Sep 2021   Prob (F-statistic):           5.25e-94
Time:                         09:10:58   Log-Likelihood:                -1476.1
No. Observations:                 2021   AIC:                             2992.
Df Residuals:                     2001   BIC:                             3104.
Df Model:                           19                                         
Covariance Type:             nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------

Analysis EVs - 
significant variables at 5% level of significance:
1. 
2. pop density 
3. 3 to floor rooms
4. income
5. electricity tariff
6. 
7. 20-39
8. 40-59
9. 60-64
10. voting: marginally for increases EV share 

not significant:
1. % participation
2. temperature
3. permanent resident population 
4. swiss_citizen
5. age_0to14%_%
6. 65 and above 
7. men
10. Numpoints

Solar PV installations

In [ ]:
endo_var = "share_pv"
exog_vars = ["percentage_population_in_detached_houses", "population_per_km2", "percentage_flats_3to4_rooms", "taxable_income_per_taxable_person_in_chf",
"mean_electricity_tarif_2021_rp_per_kwh", "% participation", "temp_mean", "pv_out_mea", "permanent_resident_population_%", "swiss_citizen_%", "age_0to14_%", "age_15to19_%", "age_20to39_%",
"age_40to59_%", "age_60to64_%", "age_65tomore_%", "men_%", "marginally for", "strongly against", "strongly for"]
ols_reg(X = df[exog_vars], y = df[endo_var])

                            OLS Regression Results                            
Dep. Variable:               share_pv   R-squared:                       0.095
Model:                            OLS   Adj. R-squared:                  0.086
Method:                 Least Squares   F-statistic:                     11.06
Date:                Fri, 03 Sep 2021   Prob (F-statistic):           2.54e-32
Time:                        09:21:22   Log-Likelihood:                 3919.4
No. Observations:                2021   AIC:                            -7799.
Df Residuals:                    2001   BIC:                            -7687.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

Analysis EVs - 
significant variables at 5% level of significance:
1. 
2. pop density 
3. 3 to floor rooms
4. income
5. electricity tariff
6. 
7. 20-39
8. 40-59
9. 60-64
10. voting: marginally for increases EV share 

not significant:
1. % participation
2. temperature
3. permanent resident population 
4. swiss_citizen
5. age_0to14%_%
6. 65 and above 
7. men

In [ ]:
#### log share_pv 
endo_var = "share_pv"
exog_vars = ["percentage_population_in_detached_houses", "population_per_km2", "percentage_flats_3to4_rooms", "taxable_income_per_taxable_person_in_chf",
"mean_electricity_tarif_2021_rp_per_kwh", "% participation", "temp_mean", "pv_out_mea", "permanent_resident_population_%", "swiss_citizen_%", "age_0to14_%", "age_15to19_%", "age_20to39_%",
"age_40to59_%", "age_60to64_%", "age_65tomore_%", "men_%", "marginally for", "strongly against", "strongly for"]
ols_reg(X = df[exog_vars], y = np.log(df[endo_var]))

                            OLS Regression Results                            
Dep. Variable:               share_pv   R-squared:                       0.194
Model:                            OLS   Adj. R-squared:                  0.186
Method:                 Least Squares   F-statistic:                     25.33
Date:                Fri, 03 Sep 2021   Prob (F-statistic):           5.85e-80
Time:                        09:15:24   Log-Likelihood:                -1995.0
No. Observations:                2021   AIC:                             4030.
Df Residuals:                    2001   BIC:                             4142.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

Analysis PVs - 
significant variables at 5% level of significance:
1. detached houses 
2. swiss_citizen 
3. 3 to 4 floor rooms
4. younger adult demographics, while older adult demographics tend to be averse. 
5. electricity tariff
6. voting: voting for increases PV share

not significant:
1. pv output however temperature is .... 


Share Renewable heating

In [ ]:
#### share_renewable_heating 
endo_var = "share_renewable_heating"
exog_vars_heat = ["percentage_population_in_detached_houses", "population_per_km2", "percentage_flats_3to4_rooms", "taxable_income_per_taxable_person_in_chf", "Heating data coverage",
"mean_electricity_tarif_2021_rp_per_kwh", "% participation", "temp_mean", "pv_out_mea", "permanent_resident_population_%", "swiss_citizen_%", "age_0to14_%", "age_15to19_%", "age_20to39_%",
"age_40to59_%", "age_60to64_%", "age_65tomore_%", "men_%", "marginally for", "strongly against", "strongly for"]
ols_reg(X = df[exog_vars], y = df[endo_var])

                               OLS Regression Results                              
Dep. Variable:     share_renewable_heating   R-squared:                       0.525
Model:                                 OLS   Adj. R-squared:                  0.521
Method:                      Least Squares   F-statistic:                     116.5
Date:                     Fri, 03 Sep 2021   Prob (F-statistic):          2.53e-306
Time:                             09:25:25   Log-Likelihood:                 1625.6
No. Observations:                     2021   AIC:                            -3211.
Df Residuals:                         2001   BIC:                            -3099.
Df Model:                               19                                         
Covariance Type:                 nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------

In [ ]:
ols_reg(X = df[exog_vars], y = np.log(df[endo_var]))

                               OLS Regression Results                              
Dep. Variable:     share_renewable_heating   R-squared:                       0.581
Model:                                 OLS   Adj. R-squared:                  0.577
Method:                      Least Squares   F-statistic:                     146.1
Date:                     Fri, 03 Sep 2021   Prob (F-statistic):               0.00
Time:                             09:25:43   Log-Likelihood:                -700.95
No. Observations:                     2021   AIC:                             1442.
Df Residuals:                         2001   BIC:                             1554.
Df Model:                               19                                         
Covariance Type:                 nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------

Analysis Heating - 
significant variables at 5% level of significance:
1. detached houses 
2. swiss_citizen 
3. 3 to 4 floor rooms
4. younger adult demographics, while older adult demographics tend to be averse. 
5. electricity tariff
6. voting: voting for increases PV share

not significant:
1. pv output however temperature is .... 